# Trading Performance Analysis

Ad-hoc exploration notebook for analyzing trading track record.

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yaml

from src import performance, slippage_analyzer

# Load configuration
with open('../config/config.yaml') as f:
    config = yaml.safe_load(f)

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data

In [ ]:
# Define date range for analysis
START_DATE = None  # e.g., '2026-01-01'
END_DATE = None    # e.g., '2026-01-31'

# Load snapshots and compute equity curve
snapshots = performance.load_snapshots(
    config['paths']['snapshots'],
    start_date=START_DATE,
    end_date=END_DATE
)

equity = performance.compute_equity_curve(snapshots)
returns = performance.compute_returns(equity)

print(f"Loaded {len(snapshots)} daily snapshots")
if not snapshots.empty:
    print(f"Date range: {snapshots.index[0]} to {snapshots.index[-1]}")

## Performance Metrics

In [ ]:
# Calculate all metrics
metrics = performance.compute_all_metrics(
    config['paths']['snapshots'],
    config['paths']['executions'],
    start_date=START_DATE,
    end_date=END_DATE
)

# Display formatted report
print(performance.format_performance_report(metrics))

## Equity Curve

In [ ]:
if not equity.empty:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    ax.plot(equity.index, equity.values, linewidth=2, color='#2E86AB')
    ax.fill_between(equity.index, equity.values, alpha=0.3, color='#2E86AB')
    
    ax.set_title('Portfolio Equity Curve', fontsize=14, fontweight='bold')
    ax.set_ylabel('Portfolio Value ($)')
    ax.set_xlabel('Date')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    plt.tight_layout()
    plt.show()
else:
    print("No equity data available")

## Drawdown Analysis

In [ ]:
if not equity.empty:
    drawdown = performance.drawdown_series(equity)
    
    fig, ax = plt.subplots(figsize=(14, 4))
    
    ax.fill_between(drawdown.index, drawdown.values * 100, 0, color='#E74C3C', alpha=0.7)
    ax.plot(drawdown.index, drawdown.values * 100, color='#C0392B', linewidth=1)
    
    ax.set_title('Drawdown', fontsize=12, fontweight='bold')
    ax.set_ylabel('Drawdown (%)')
    ax.set_xlabel('Date')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Max Drawdown: {performance.max_drawdown(equity)*100:.2f}%")
    print(f"Max Drawdown Duration: {performance.max_drawdown_duration(equity)} days")

## Rolling Sharpe Ratio

In [ ]:
if len(returns) >= 30:
    rolling_sharpe = performance.rolling_sharpe(returns, window=30)
    
    fig, ax = plt.subplots(figsize=(14, 4))
    
    ax.plot(rolling_sharpe.index, rolling_sharpe.values, linewidth=2, color='#27AE60')
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
    ax.axhline(y=1, color='green', linestyle='--', alpha=0.5, label='Sharpe = 1')
    ax.axhline(y=2, color='blue', linestyle='--', alpha=0.5, label='Sharpe = 2')
    
    ax.set_title('30-Day Rolling Sharpe Ratio', fontsize=12, fontweight='bold')
    ax.set_ylabel('Sharpe Ratio')
    ax.set_xlabel('Date')
    ax.legend()
    
    plt.tight_layout()
    plt.show()
else:
    print("Need at least 30 days of data for rolling Sharpe")

## Slippage Analysis

In [ ]:
# Run slippage analysis
slippage_data = slippage_analyzer.analyze_slippage(
    config['paths']['executions'],
    start_date=START_DATE,
    end_date=END_DATE
)

print(slippage_analyzer.format_slippage_report(slippage_data))

In [ ]:
# Slippage by symbol
if not slippage_data['by_symbol'].empty:
    display(slippage_data['by_symbol'])

In [ ]:
# Slippage distribution histogram
executions = slippage_data['executions']

if not executions.empty and 'slippage_bps' in executions.columns:
    slippage_values = executions['slippage_bps'].dropna()
    
    if not slippage_values.empty:
        fig, ax = plt.subplots(figsize=(10, 5))
        
        ax.hist(slippage_values, bins=30, color='#3498DB', edgecolor='white', alpha=0.7)
        ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero slippage')
        ax.axvline(x=slippage_values.mean(), color='green', linestyle='--', linewidth=2, label=f'Mean: {slippage_values.mean():.1f} bps')
        
        ax.set_title('Slippage Distribution', fontsize=12, fontweight='bold')
        ax.set_xlabel('Slippage (bps)')
        ax.set_ylabel('Frequency')
        ax.legend()
        
        plt.tight_layout()
        plt.show()

## Trade Analysis

In [ ]:
# Compute trade P&L
executions = slippage_data['executions']
trades = performance.compute_trade_pnl(executions)

if not trades.empty:
    print(f"Total trades: {len(trades)}")
    print(f"Win rate: {performance.win_rate(trades)*100:.1f}%")
    print(f"Profit factor: {performance.profit_factor(trades):.2f}")
    print(f"\nTotal P&L: ${trades['net_pnl'].sum():,.2f}")
    print(f"Average win: ${trades[trades['net_pnl'] > 0]['net_pnl'].mean():,.2f}")
    print(f"Average loss: ${trades[trades['net_pnl'] < 0]['net_pnl'].mean():,.2f}")
else:
    print("No completed trades found")

In [ ]:
# P&L distribution
if not trades.empty:
    fig, ax = plt.subplots(figsize=(10, 5))
    
    colors = ['#27AE60' if x > 0 else '#E74C3C' for x in trades['net_pnl']]
    ax.bar(range(len(trades)), trades['net_pnl'].values, color=colors, alpha=0.7)
    
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.set_title('Trade P&L Distribution', fontsize=12, fontweight='bold')
    ax.set_xlabel('Trade Number')
    ax.set_ylabel('Net P&L ($)')
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
    
    plt.tight_layout()
    plt.show()

## Daily Returns Distribution

In [ ]:
if not returns.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    axes[0].hist(returns * 100, bins=30, color='#3498DB', edgecolor='white', alpha=0.7)
    axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2)
    axes[0].set_title('Daily Returns Distribution', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Daily Return (%)')
    axes[0].set_ylabel('Frequency')
    
    # Monthly returns heatmap-style bar chart
    monthly_returns = returns.resample('M').apply(lambda x: (1 + x).prod() - 1) * 100
    colors = ['#27AE60' if x > 0 else '#E74C3C' for x in monthly_returns]
    axes[1].bar(monthly_returns.index.strftime('%Y-%m'), monthly_returns.values, color=colors, alpha=0.7)
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[1].set_title('Monthly Returns', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Month')
    axes[1].set_ylabel('Return (%)')
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    # Statistics
    print(f"Daily return statistics:")
    print(f"  Mean: {returns.mean()*100:.3f}%")
    print(f"  Std: {returns.std()*100:.3f}%")
    print(f"  Skewness: {returns.skew():.3f}")
    print(f"  Kurtosis: {returns.kurtosis():.3f}")

## Custom Analysis

Add your own analysis cells below.

In [ ]:
# Your custom analysis here